In [1]:
import os
from pathlib import Path

current_directory = Path.cwd()
if current_directory.name == "MT":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory

os.chdir(root_directory)
root_directory

PosixPath('/home/wueesmat/projects/anomalib')

In [145]:
import numpy as np
import pandas as pd

# Import the required modules
from anomalib.data import MVTec
from anomalib.data.image.mvtec import make_mvtec_dataset
from anomalib import TaskType
from anomalib.data.utils import LabelName

In [ ]:
from lightning.pytorch import seed_everything
seed_everything(42, workers=True)

In [55]:
root = Path('./datasets/MVTec')
category = 'carpet' #'bottle'
path = root / category
path

PosixPath('datasets/MVTec/carpet')

In [56]:
#samples = make_mvtec_dataset(path, split='train', split_ratio=0.1, seed=0)
samples = make_mvtec_dataset(path, split=None)
samples.head()

,path,split,label,image_path,label_index,mask_path
0,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
1,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
2,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
3,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
4,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...


In [38]:
samples.shape[0]

292

In [ ]:
# function call
self.samples = make_mvtec_dataset_contaminated(self.root_category, split=self.split, extensions=IMG_EXTENSIONS, contamination_ratio=0.0)

In [146]:
datamodule = MVTec(category="carpet")

In [147]:
datamodule

In [148]:
datamodule.setup

<bound method AnomalibDataModule.setup of <anomalib.data.image.mvtec.MVTec object at 0x7fc8db523730>>

In [ ]:
datamodule = MVTec_contaminated(category="carpet", cont_ratio=0.0, cont_ratio_max=0.0)

In [ ]:
self.train_data = MVTecDataset_contaminated(
            task=self.task,
            transform=self.train_transform,
            split=Split.TRAIN,
            root=self.root,
            category=self.category,
            cont_ratio=self.cont_ratio, 
            cont_ratio_max=self.cont_ratio_max
        )

In [ ]:
class MVTec_contaminated(MVTec):
   
    def __init__(
        self,
        root: Path | str = "./datasets/MVTec",
        category: str = "bottle",
        train_batch_size: int = 32,
        eval_batch_size: int = 32,
        num_workers: int = 8,
        task: TaskType | str = TaskType.SEGMENTATION,
        image_size: tuple[int, int] | None = None,
        transform: Transform | None = None,
        train_transform: Transform | None = None,
        eval_transform: Transform | None = None,
        test_split_mode: TestSplitMode | str = TestSplitMode.FROM_DIR,
        test_split_ratio: float = 0.2,
        val_split_mode: ValSplitMode | str = ValSplitMode.SAME_AS_TEST,
        val_split_ratio: float = 0.5,
        seed: int | None = None,
        cont_ratio: float=0.0,
        cont_ratio_max: float=0.0,
        run: int=1
    ) -> None:
        super().__init__(
            train_batch_size=train_batch_size,
            eval_batch_size=eval_batch_size,
            image_size=image_size,
            transform=transform,
            train_transform=train_transform,
            eval_transform=eval_transform,
            num_workers=num_workers,
            test_split_mode=test_split_mode,
            test_split_ratio=test_split_ratio,
            val_split_mode=val_split_mode,
            val_split_ratio=val_split_ratio,
            seed=seed,
            task=task,
            root=root,
            category=category            
        )

        self.cont_ratio = cont_ratio
        self.cont_ratio_max = cont_ratio_max
        self.run = run

    def _setup(self, _stage: str | None = None) -> None:
        """Set up the datasets and perform dynamic subset splitting.

        This method may be overridden in subclass for custom splitting behaviour.

        Note:
            The stage argument is not used here. This is because, for a given instance of an AnomalibDataModule
            subclass, all three subsets are created at the first call of setup(). This is to accommodate the subset
            splitting behaviour of anomaly tasks, where the validation set is usually extracted from the test set, and
            the test set must therefore be created as early as the `fit` stage.

        """
        self.train_data = MVTecDataset_contaminated(
            task=self.task,
            transform=self.train_transform,
            split=Split.TRAIN,
            root=self.root,
            category=self.category,
            cont_ratio=self.cont_ratio,
            cont_ratio_max=self.cont_ratio_max,
            run=self.run            
        )
        self.test_data = MVTecDataset_contaminated(
            task=self.task,
            transform=self.eval_transform,
            split=Split.TEST,
            root=self.root,
            category=self.category,
            cont_ratio=self.cont_ratio,
            cont_ratio_max=self.cont_ratio_max,
            run=self.run   
        )


In [ ]:
class MVTecDataset_contaminated(MVTecDataset):

    def __init__(
        self,
        task: TaskType,
        root: Path | str = "./datasets/MVTec",
        category: str = "bottle",
        transform: Transform | None = None,
        split: str | Split | None = None,
        cont_ratio: float=0.0,
        cont_ratio_max: float=0.0,
        run: int=1
    ) -> None:
        super().__init__(task=task, 
                         root=root, 
                         category=category, 
                         transform=transform, 
                         split=split)
        
        self.cont_ratio = cont_ratio
        self.cont_ratio_max = cont_ratio_max
        self.run = run

        self.samples = make_mvtec_dataset_contaminated(self.root_category, 
                                                       split=self.split, 
                                                       extensions=IMG_EXTENSIONS, 
                                                       cont_ratio=self.cont_ratio, 
                                                       cont_ratio_max=self.cont_ratio_max,
                                                       run=self.run)
        



In [ ]:
def make_mvtec_dataset_contaminated(
    root: str | Path,
    split: str | Split | None = None,
    extensions: Sequence[str] | None = None,
    cont_ratio: float=0.0,
    cont_ratio_max: float=0.0,
    run: int=1
) -> DataFrame:   

    ## Original dataset
    samples_original = make_mvtec_dataset(path, split=split, extensions=extensions)
    samples_original_train = samples_original[(samples_original["split"]=="train")]
    samples_original_test_normal = samples_original[(samples_original["split"]=="test") & (samples_original["label_index"]==LabelName.NORMAL)]
    samples_original_test_abnormal = samples_original[(samples_original["split"]=="test") & (samples_original["label_index"]==LabelName.ABNORMAL)]
    n_original_train = samples_original_train.shape[0]

    ## Contaminated dataset

    # create test sets
    contamination_ratio_max = cont_ratio_max
    n_swap = int(n_original_train/(1.0-contamination_ratio_max) - n_original_train)
    samples_contaminated_swap = samples_original_test_abnormal.sample(n=n_swap, axis=0, replace=False, random_state=run)

    samples_contaminated_test_normal = samples_original_test_normal
    samples_contaminated_test_abnormal = samples_original_test_abnormal.drop(samples_contaminated_swap.index)
    samples_contaminated_test = pd.concat([samples_contaminated_test_normal, samples_contaminated_test_abnormal], axis=0, ignore_index=True)

    # create contaminated training sets
    contamination_ratio = cont_ratio
    n_contamination = int(contamination_ratio/(contamination_ratio_max+1e6) * n_swap)

    samples_contaminated_train_normal = samples_original_train
    samples_contaminated_train_abnormal = samples_contaminated_swap.iloc[0:n_contamination,:] 
    samples_contaminated_train = pd.concat([samples_contaminated_train_normal, samples_contaminated_train_abnormal], axis=0, ignore_index=True)

    # Return required samples
    if split==None:
        samples = pd.concat([samples_contaminated_train, samples_contaminated_test], axis=0, ignore_index=True)
    elif split=="train":
        samples = samples_contaminated_train
    elif split=="test":
        samples = samples_contaminated_test
    else:
        samples= samples.head(0)

    return samples





In [169]:
df_empty = df.head(0)

NameError: name 'df' is not defined

In [173]:
test = samples.head(0)
test

,path,split,label,image_path,label_index,mask_path


In [168]:
samples_original = make_mvtec_dataset(path, split="asfasd")
samples_original

,path,split,label,image_path,label_index,mask_path


In [158]:
## Original dataset
samples_original = make_mvtec_dataset(path, split=None)
samples_original_train = samples_original[(samples_original["split"]=="train")]
samples_original_test_normal = samples_original[(samples_original["split"]=="test") & (samples_original["label_index"]==LabelName.NORMAL)]
samples_original_test_abnormal = samples_original[(samples_original["split"]=="test") & (samples_original["label_index"]==LabelName.ABNORMAL)]

n_original = samples_original.shape[0]
n_original_train = samples_original_train.shape[0]
n_original_test_normal = samples_original_test_normal.shape[0]
n_original_test_abnormal = samples_original_test_abnormal.shape[0]

print("--------------------------------")
print(n_original)
print(n_original_train)
print(n_original_test_normal)
print(n_original_test_abnormal)
print("--------------------------------")



## Contaminated dataset

random_state_run = 1

# For all contamination ratios
contamination_ratio_max = 0.1
n_swap = int(n_original_train/(1.0-contamination_ratio_max) - n_original_train)
samples_contaminated_swap = samples_original_test_abnormal.sample(n=n_swap, axis=0, replace=False, random_state=random_state_run)

samples_contaminated_test_normal = samples_original_test_normal
samples_contaminated_test_abnormal = samples_original_test_abnormal.drop(samples_contaminated_swap.index)
samples_contaminated_test = pd.concat([samples_contaminated_test_normal, samples_contaminated_test_abnormal], axis=0, ignore_index=True)

n_contaminated_test_normal = samples_contaminated_test_normal.shape[0]
n_contaminated_test_abnormal = samples_contaminated_test_abnormal.shape[0]
n_contaminated_test = samples_contaminated_test.shape[0]

# For given contamination ratio
contamination_ratio = 0.1
n_contamination = int(contamination_ratio/(contamination_ratio_max+1e6) * n_swap)

samples_contaminated_train_normal = samples_original_train
samples_contaminated_train_abnormal = samples_contaminated_swap.iloc[0:n_contamination,:] 
samples_contaminated_train = pd.concat([samples_contaminated_train_normal, samples_contaminated_train_abnormal], axis=0, ignore_index=True)

n_contaminated_train_normal = samples_contaminated_train_normal.shape[0]
n_contaminated_train_abnormal = samples_contaminated_train_abnormal.shape[0]
n_contaminated_train = samples_contaminated_train.shape[0]



print("--------------------------------")
print(n_contaminated_train_normal)
print(n_contaminated_train_abnormal)
print(n_contaminated_train)
print(n_contaminated_test_normal)
print(n_contaminated_test_abnormal)
print(n_contaminated_test)
print("--------------------------------")




--------------------------------
397
280
28
89
--------------------------------
--------------------------------
280
0
280
28
58
86
--------------------------------


In [120]:
samples_contaminated_train_abnormal = samples_contaminated_swap.iloc[0:n_contamination,:]

In [122]:
samples_contaminated_swap.shape[0]

31

In [123]:
samples_contaminated_train_abnormal.shape[0]

15

In [117]:
samples_contaminated_train

,path,split,label,image_path,label_index,mask_path
0,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
1,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
2,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
3,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
4,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
...,...,...,...,...,...,...
290,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
291,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
292,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
293,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,metal_contamination,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...


In [118]:
samples_contaminated_train[samples_contaminated_train.label=="good"]

,path,split,label,image_path,label_index,mask_path
0,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
1,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
2,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
3,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
4,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
...,...,...,...,...,...,...
275,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
276,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
277,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
278,/home/wueesmat/projects/anomalib/datasets/MVTe...,train,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,


In [116]:
samples_contaminated_test

,path,split,label,image_path,label_index,mask_path
0,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
1,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
2,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
3,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
4,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,good,/home/wueesmat/projects/anomalib/datasets/MVTe...,0,
...,...,...,...,...,...,...
81,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
82,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
83,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
84,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...


In [107]:
samples_contaminated_swap

,path,split,label,image_path,label_index,mask_path
72,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,hole,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
35,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,cut,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
112,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
0,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
11,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
14,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
87,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,metal_contamination,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
3,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,color,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
88,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,metal_contamination,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...
109,/home/wueesmat/projects/anomalib/datasets/MVTe...,test,thread,/home/wueesmat/projects/anomalib/datasets/MVTe...,1,/home/wueesmat/projects/anomalib/datasets/MVTe...


31

In [ ]:
samples_contaminated_swap.shape[0]

In [35]:
samples_contaminated_train_normal = samples_original_train + xxx_part
samples_contaminated_test_normal = samples_original_test_normal
samples_contaminated_test_abnormal = samples_original_test_abnormal - xxx_complete

path           /home/wueesmat/projects/anomalib/datasets/MVTe...
split                                                       test
label                                               broken_large
image_path     /home/wueesmat/projects/anomalib/datasets/MVTe...
label_index                                                    1
mask_path      /home/wueesmat/projects/anomalib/datasets/MVTe...
Name: 0, dtype: object

In [16]:
datamodule = MVTec(category="carpet")